# Train and Evaluate Model

In [1]:
# Import packages
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

tf.__version__ # 2.x

'2.7.0'

## Model

In [2]:
# Sequence model
seq_model = tf.keras.Sequential()
seq_model.add(tf.keras.Input(shape=(784,), name="input_x"))
seq_model.add(layers.Dense(64, activation="relu", name="dense_1"))
seq_model.add(layers.Dense(64, activation="relu", name="dense_2"))
seq_model.add(layers.Dense(10, activation="softmax", name="y"))

# View
seq_model.summary()

2021-12-05 09:28:21.390083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 y (Dense)                   (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


2021-12-05 09:28:21.394217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 09:28:21.394665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 09:28:21.395692: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-05 09:28:21.396381: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [3]:
# Functional model
inputs = tf.keras.Input(shape=(784,), name="x")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
y = layers.Dense(10, activation="softmax", name="y")(x)

# Wrap layers into a model object
func_model = tf.keras.Model(inputs=inputs, outputs=y)

# View
func_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x (InputLayer)              [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 y (Dense)                   (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Plot sequence model
tf.keras.utils.plot_model(seq_model, "/tmp/seq_model.png", show_shapes=True, show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [5]:
# Plot functional model
tf.keras.utils.plot_model(func_model, "/tmp/func_model.png", show_shapes=True, show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Train and Evaluate

Typical end-to-end workflow consists of:

- Training
- Validation on a holdout set generated from the original training data
- Evaluation on the test data

We'll use very famous MNIST data to solve a digit image classification problem.
Both models, sequence and functonal models can be used in a similar manner.

In [6]:
# Load data into memory
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reshape to a vector from matrix of 28*28 and normalize it
x_train = x_train.reshape(60000, 784).astype("float32") / 255
y_train = y_train.astype("float32")

x_test = x_test.reshape(10000, 784).astype("float32") / 255
y_test = y_test.astype("float32")

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2020)

print("Train:", x_train.shape, y_train.shape)
print("Test:", x_test.shape, y_test.shape)
print("Validate:", x_val.shape, y_val.shape)

Train: (48000, 784) (48000,)
Test: (10000, 784) (10000,)
Validate: (12000, 784) (12000,)


Specify the training configuration (optimizer, loss, metrics) using `compile()`

In [7]:
# Compile model
seq_model.compile(
    # Model optimizer
    optimizer = tf.keras.optimizers.RMSprop(),
    
    # Model loss function
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    
    # Model performance metric
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [8]:
# Model training
history = seq_model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=10,
    validation_data=(x_val, y_val),
)

Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 0.3444 - sparse_categorical_accuracy: 0.9028 - val_loss: 0.2028 - val_sparse_categorical_accuracy: 0.9406
Epoch 2/10
750/750 [==============================] - 2s 2ms/step - loss: 0.1623 - sparse_categorical_accuracy: 0.9530 - val_loss: 0.1514 - val_sparse_categorical_accuracy: 0.9581
Epoch 3/10
750/750 [==============================] - 2s 2ms/step - loss: 0.1177 - sparse_categorical_accuracy: 0.9656 - val_loss: 0.1252 - val_sparse_categorical_accuracy: 0.9631
Epoch 4/10
750/750 [==============================] - 2s 2ms/step - loss: 0.0939 - sparse_categorical_accuracy: 0.9713 - val_loss: 0.1100 - val_sparse_categorical_accuracy: 0.9683
Epoch 5/10
750/750 [==============================] - 2s 2ms/step - loss: 0.0775 - sparse_categorical_accuracy: 0.9760 - val_loss: 0.1127 - val_sparse_categorical_accuracy: 0.9676
Epoch 6/10
750/750 [==============================] - 2s 2ms/step - loss: 0.0659 - sparse_categorica

Evaluate model on the test dataset using `evaluate()`

In [9]:
results = seq_model.evaluate(x_test, y_test, batch_size=128)
print("Test loss, Test acc:", results)

79/79 [==============================] - 0s 1ms/step - loss: 0.0924 - sparse_categorical_accuracy: 0.9767
Test loss, Test acc: [0.09241479635238647, 0.9767000079154968]


Generate predictions (probabilities) on test set using `predict()`

In [10]:
predictions = seq_model.predict(x_test)
print("Predictions:", predictions.shape)
print("First prediction:", predictions[0])

Predictions: (10000, 10)
First prediction: [9.1651220e-09 5.2436519e-15 4.4976437e-06 3.1977266e-05 4.8915404e-14
 2.4661487e-08 4.8377372e-13 9.9996328e-01 1.5918646e-08 1.6914754e-07]
